In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [5]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels = zip(*data)

## Inspect label counts

In [6]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [7]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels = zip(*balanced_data)

In [8]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


# Prepare 10 cross validation folds

In [9]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(train_index_set)),
    }


# Load features

In [10]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [11]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [12]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [13]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [14]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [15]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [17]:
#class LangIdConvNet(nn.Module):
#    def __init__(self):
#        super(LangIdConvNet, self).__init__()
#        
#        self.lstm = nn.LSTM(input_size=512, hidden_size=3, num_layers=1, batch_first=True)
#        # self.gru = nn.GRU(input_size=512, hidden_size=3, num_layers=1, batch_first=True)
#        self.drop1 = nn.Dropout(p=0.3)
#        self.lin1 = nn.Linear(in_features=3, out_features=3)
        
        
        
#    def forward(self, x):
#        output, (h_n, c_n) = self.lstm(x)
#        # output, h_n = self.gru(x)
#        x = output[:, -1, :] # [all batches, last time step, all neurons]
#        
#        x = self.drop1(x)
#        x = self.lin1(x)
#        
#        return x

class LangIdRNN(nn.Module):
    def __init__(self):
        super(LangIdRNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        # x = self.drop2(x)
        x = self.lin2(x)
        
        return x

# Train Classification Models

In [18]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    
    return train_x, train_y, test_x, test_y

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y = get_data_for_fold(fold_id, feature_name, batch_size)
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [19]:
def train_on_fold(fold_id, feature_name, batch_size, epochs):
    device = torch.device("cuda:0")
    torch.manual_seed(0)
    results = {}
    
    model = LangIdRNN().to(device)

    train_loader, test_loader = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(LangIdRNN(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc
        }
        
    return results
    

In [20]:
import csv
from pathlib import Path

def save_results(all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        Path('lstm_results_3').mkdir(exist_ok=True)
        fname = f"lstm_results_3/{feature_name}_{fold_index}.csv"
        with open(fname, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(
                ['epoch', 'train_loss', 'test_loss', 'train_acc', 'test_acc']
            )
            
            for epoch in sorted(result_entry['epochs'].keys()):
                writer.writerow([
                    epoch,
                    result_entry['epochs'][epoch]['train_loss'],
                    result_entry['epochs'][epoch]['test_loss'],
                    result_entry['epochs'][epoch]['train_acc'],
                    result_entry['epochs'][epoch]['test_acc']
                ])

In [21]:
all_results = []

for fold_index in cv_folds:
    for feature_name in raw_features:
        resutls = train_on_fold(fold_index, feature_name, batch_size=100, epochs=1000)
        all_results.append({
            'fold_index': fold_index,
            'feature_name': feature_name,
            'epochs': resutls
        })
        save_results(all_results)

---------------------------------------------------------------------------------
      Layer (type)                  Output Shape         Param #     Tr. Param #
          Conv1d-1                 [10, 1, 2996]           1,537           1,537
         Dropout-2                 [10, 1, 2996]               0               0
       MaxPool1d-3                 [10, 1, 1498]               0               0
             GRU-4     [10, 1498, 3], [1, 10, 3]              54              54
          Linear-5                       [10, 3]              12              12
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
---------------------------------------------------------------------------------
Epoch: 10. Train Loss: 1.136. Test Loss: 1.134. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 20. Train Loss: 1.108. Test Loss: 1.11. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 30. Train Loss: 1.086. Test Loss: 1.085. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 40. Train Loss: 1.063. Test

Epoch: 900. Train Loss: 0.109. Test Loss: 0.09731. Train Acc: 1.0. Test Acc:1.0
Epoch: 910. Train Loss: 0.1274. Test Loss: 0.09517. Train Acc: 0.9804. Test Acc:1.0
Epoch: 920. Train Loss: 0.09547. Test Loss: 0.09457. Train Acc: 1.0. Test Acc:1.0
Epoch: 930. Train Loss: 0.1079. Test Loss: 0.0918. Train Acc: 1.0. Test Acc:1.0
Epoch: 940. Train Loss: 0.09228. Test Loss: 0.09346. Train Acc: 1.0. Test Acc:1.0
Epoch: 950. Train Loss: 0.1051. Test Loss: 0.08986. Train Acc: 0.9804. Test Acc:1.0
Epoch: 960. Train Loss: 0.09942. Test Loss: 0.08973. Train Acc: 1.0. Test Acc:1.0
Epoch: 970. Train Loss: 0.1092. Test Loss: 0.08764. Train Acc: 1.0. Test Acc:1.0
Epoch: 980. Train Loss: 0.1054. Test Loss: 0.08646. Train Acc: 1.0. Test Acc:1.0
Epoch: 990. Train Loss: 0.09427. Test Loss: 0.0836. Train Acc: 1.0. Test Acc:1.0
Epoch: 1000. Train Loss: 0.08409. Test Loss: 0.08392. Train Acc: 1.0. Test Acc:1.0
---------------------------------------------------------------------------------
      Layer (type)

Epoch: 790. Train Loss: 0.1929. Test Loss: 0.1178. Train Acc: 0.9608. Test Acc:1.0
Epoch: 800. Train Loss: 0.1216. Test Loss: 0.1165. Train Acc: 1.0. Test Acc:1.0
Epoch: 810. Train Loss: 0.1678. Test Loss: 0.1111. Train Acc: 0.9804. Test Acc:1.0
Epoch: 820. Train Loss: 0.1126. Test Loss: 0.1112. Train Acc: 1.0. Test Acc:1.0
Epoch: 830. Train Loss: 0.1166. Test Loss: 0.1103. Train Acc: 1.0. Test Acc:1.0
Epoch: 840. Train Loss: 0.2053. Test Loss: 0.1078. Train Acc: 0.9804. Test Acc:1.0
Epoch: 850. Train Loss: 0.1296. Test Loss: 0.106. Train Acc: 1.0. Test Acc:1.0
Epoch: 860. Train Loss: 0.1141. Test Loss: 0.1058. Train Acc: 1.0. Test Acc:1.0
Epoch: 870. Train Loss: 0.1098. Test Loss: 0.09992. Train Acc: 1.0. Test Acc:1.0
Epoch: 880. Train Loss: 0.1217. Test Loss: 0.1001. Train Acc: 1.0. Test Acc:1.0
Epoch: 890. Train Loss: 0.1093. Test Loss: 0.09692. Train Acc: 1.0. Test Acc:1.0
Epoch: 900. Train Loss: 0.1153. Test Loss: 0.09384. Train Acc: 1.0. Test Acc:1.0
Epoch: 910. Train Loss: 0.115

Epoch: 680. Train Loss: 0.1613. Test Loss: 0.1394. Train Acc: 1.0. Test Acc:1.0
Epoch: 690. Train Loss: 0.1672. Test Loss: 0.1359. Train Acc: 1.0. Test Acc:1.0
Epoch: 700. Train Loss: 0.1573. Test Loss: 0.1338. Train Acc: 1.0. Test Acc:1.0
Epoch: 710. Train Loss: 0.1465. Test Loss: 0.1303. Train Acc: 1.0. Test Acc:1.0
Epoch: 720. Train Loss: 0.1297. Test Loss: 0.1283. Train Acc: 1.0. Test Acc:1.0
Epoch: 730. Train Loss: 0.1258. Test Loss: 0.1259. Train Acc: 1.0. Test Acc:1.0
Epoch: 740. Train Loss: 0.1261. Test Loss: 0.1232. Train Acc: 1.0. Test Acc:1.0
Epoch: 750. Train Loss: 0.135. Test Loss: 0.1221. Train Acc: 1.0. Test Acc:1.0
Epoch: 760. Train Loss: 0.1254. Test Loss: 0.1185. Train Acc: 1.0. Test Acc:1.0
Epoch: 770. Train Loss: 0.1238. Test Loss: 0.1169. Train Acc: 1.0. Test Acc:1.0
Epoch: 780. Train Loss: 0.1157. Test Loss: 0.1142. Train Acc: 1.0. Test Acc:1.0
Epoch: 790. Train Loss: 0.1632. Test Loss: 0.1127. Train Acc: 0.9608. Test Acc:1.0
Epoch: 800. Train Loss: 0.1106. Test L

Epoch: 570. Train Loss: 0.1915. Test Loss: 0.184. Train Acc: 1.0. Test Acc:1.0
Epoch: 580. Train Loss: 0.203. Test Loss: 0.1779. Train Acc: 1.0. Test Acc:1.0
Epoch: 590. Train Loss: 0.1957. Test Loss: 0.1762. Train Acc: 0.9804. Test Acc:1.0
Epoch: 600. Train Loss: 0.1835. Test Loss: 0.168. Train Acc: 1.0. Test Acc:1.0
Epoch: 610. Train Loss: 0.1921. Test Loss: 0.1655. Train Acc: 0.9804. Test Acc:1.0
Epoch: 620. Train Loss: 0.1898. Test Loss: 0.1609. Train Acc: 0.9804. Test Acc:1.0
Epoch: 630. Train Loss: 0.1648. Test Loss: 0.1572. Train Acc: 1.0. Test Acc:1.0
Epoch: 640. Train Loss: 0.1794. Test Loss: 0.1528. Train Acc: 1.0. Test Acc:1.0
Epoch: 650. Train Loss: 0.1703. Test Loss: 0.1493. Train Acc: 1.0. Test Acc:1.0
Epoch: 660. Train Loss: 0.153. Test Loss: 0.1459. Train Acc: 1.0. Test Acc:1.0
Epoch: 670. Train Loss: 0.1992. Test Loss: 0.1464. Train Acc: 0.9804. Test Acc:1.0
Epoch: 680. Train Loss: 0.1597. Test Loss: 0.1382. Train Acc: 1.0. Test Acc:1.0
Epoch: 690. Train Loss: 0.1666. 

...

Epoch: 10. Train Loss: 1.118. Test Loss: 1.116. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 20. Train Loss: 1.088. Test Loss: 1.092. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 30. Train Loss: 1.071. Test Loss: 1.068. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 40. Train Loss: 1.052. Test Loss: 1.045. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 50. Train Loss: 1.028. Test Loss: 1.024. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 60. Train Loss: 1.001. Test Loss: 1.002. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 70. Train Loss: 0.9831. Test Loss: 0.9802. Train Acc: 0.3529. Test Acc:0.3725
Epoch: 80. Train Loss: 0.9728. Test Loss: 0.9593. Train Acc: 0.4314. Test Acc:0.3922
Epoch: 90. Train Loss: 0.9616. Test Loss: 0.9386. Train Acc: 0.4902. Test Acc:0.5098
Epoch: 100. Train Loss: 0.9111. Test Loss: 0.9181. Train Acc: 0.6471. Test Acc:0.6471
Epoch: 110. Train Loss: 0.9135. Test Loss: 0.8976. Train Acc: 0.6471. Test Acc:0.7647
Epoch: 120. Train Loss: 0.8898. Test Loss: 0.8772. Train Acc: 0.7451. Test 

Epoch: 1000. Train Loss: 0.07759. Test Loss: 0.07493. Train Acc: 1.0. Test Acc:1.0
---------------------------------------------------------------------------------
      Layer (type)                  Output Shape         Param #     Tr. Param #
          Conv1d-1                 [10, 1, 2996]           1,537           1,537
         Dropout-2                 [10, 1, 2996]               0               0
       MaxPool1d-3                 [10, 1, 1498]               0               0
             GRU-4     [10, 1498, 3], [1, 10, 3]              54              54
          Linear-5                       [10, 3]              12              12
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
---------------------------------------------------------------------------------
Epoch: 10. Train Loss: 1.129. Test Loss: 1.127. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 20. Train Loss: 1.109. Test Loss: 1.112. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 30. Train Loss: 1.095. Tes

Epoch: 890. Train Loss: 0.2656. Test Loss: 0.4677. Train Acc: 0.9216. Test Acc:0.8039
Epoch: 900. Train Loss: 0.1711. Test Loss: 0.1638. Train Acc: 0.9608. Test Acc:0.9608
Epoch: 910. Train Loss: 0.1319. Test Loss: 0.109. Train Acc: 0.9804. Test Acc:1.0
Epoch: 920. Train Loss: 0.1189. Test Loss: 0.1378. Train Acc: 1.0. Test Acc:0.9804
Epoch: 930. Train Loss: 0.1128. Test Loss: 0.1288. Train Acc: 1.0. Test Acc:0.9804
Epoch: 940. Train Loss: 0.114. Test Loss: 0.1191. Train Acc: 1.0. Test Acc:0.9804
Epoch: 950. Train Loss: 0.1402. Test Loss: 0.1202. Train Acc: 0.9608. Test Acc:0.9804
Epoch: 960. Train Loss: 0.1154. Test Loss: 0.1122. Train Acc: 0.9804. Test Acc:1.0
Epoch: 970. Train Loss: 0.1107. Test Loss: 0.1142. Train Acc: 0.9804. Test Acc:0.9804
Epoch: 980. Train Loss: 0.09441. Test Loss: 0.11. Train Acc: 1.0. Test Acc:1.0
Epoch: 990. Train Loss: 0.1013. Test Loss: 0.09475. Train Acc: 1.0. Test Acc:1.0
Epoch: 1000. Train Loss: 0.09591. Test Loss: 0.09154. Train Acc: 1.0. Test Acc:1.0


Epoch: 780. Train Loss: 0.112. Test Loss: 0.1186. Train Acc: 1.0. Test Acc:1.0
Epoch: 790. Train Loss: 0.1342. Test Loss: 0.1134. Train Acc: 1.0. Test Acc:1.0
Epoch: 800. Train Loss: 0.1063. Test Loss: 0.1116. Train Acc: 1.0. Test Acc:1.0
Epoch: 810. Train Loss: 0.1153. Test Loss: 0.1084. Train Acc: 1.0. Test Acc:1.0
Epoch: 820. Train Loss: 0.1045. Test Loss: 0.1081. Train Acc: 1.0. Test Acc:1.0
Epoch: 830. Train Loss: 0.1054. Test Loss: 0.1194. Train Acc: 1.0. Test Acc:1.0
Epoch: 840. Train Loss: 0.1162. Test Loss: 0.1036. Train Acc: 0.9804. Test Acc:1.0
Epoch: 850. Train Loss: 0.102. Test Loss: 0.11. Train Acc: 1.0. Test Acc:1.0
Epoch: 860. Train Loss: 0.1066. Test Loss: 0.1004. Train Acc: 1.0. Test Acc:1.0
Epoch: 870. Train Loss: 0.09645. Test Loss: 0.1029. Train Acc: 1.0. Test Acc:1.0
Epoch: 880. Train Loss: 0.09795. Test Loss: 0.09832. Train Acc: 1.0. Test Acc:1.0
Epoch: 890. Train Loss: 0.103. Test Loss: 0.09698. Train Acc: 1.0. Test Acc:1.0
Epoch: 900. Train Loss: 0.09708. Test 

Epoch: 670. Train Loss: 0.1569. Test Loss: 0.14. Train Acc: 1.0. Test Acc:1.0
Epoch: 680. Train Loss: 0.1536. Test Loss: 0.1387. Train Acc: 1.0. Test Acc:1.0
Epoch: 690. Train Loss: 0.1562. Test Loss: 0.1349. Train Acc: 1.0. Test Acc:1.0
Epoch: 700. Train Loss: 0.152. Test Loss: 0.134. Train Acc: 1.0. Test Acc:1.0
Epoch: 710. Train Loss: 0.175. Test Loss: 0.137. Train Acc: 0.9804. Test Acc:1.0
Epoch: 720. Train Loss: 0.1274. Test Loss: 0.1289. Train Acc: 1.0. Test Acc:1.0
Epoch: 730. Train Loss: 0.1248. Test Loss: 0.1254. Train Acc: 1.0. Test Acc:1.0
Epoch: 740. Train Loss: 0.124. Test Loss: 0.1248. Train Acc: 1.0. Test Acc:1.0
Epoch: 750. Train Loss: 0.1287. Test Loss: 0.117. Train Acc: 1.0. Test Acc:1.0
Epoch: 760. Train Loss: 0.1221. Test Loss: 0.1532. Train Acc: 1.0. Test Acc:0.9804
Epoch: 770. Train Loss: 0.1223. Test Loss: 0.1142. Train Acc: 1.0. Test Acc:1.0
Epoch: 780. Train Loss: 0.117. Test Loss: 0.1159. Train Acc: 1.0. Test Acc:1.0
Epoch: 790. Train Loss: 0.1531. Test Loss: 

In [22]:
LangIdRNN()

LangIdRNN(
  (conv1): Conv1d(512, 1, kernel_size=(3,), stride=(1,))
  (drop1): Dropout(p=0.3, inplace=False)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gru): GRU(1, 3, batch_first=True)
  (lin2): Linear(in_features=3, out_features=3, bias=True)
)

In [23]:
raw_features.keys()

dict_keys(['wav2vec_features-c', 'wav2vec_features-z', 'retrained-wav2vec_features-c', 'retrained-wav2vec_features-z'])